## Implementeer elk gemaakt ETL-schema in Python

**imports & connection**

In [1]:
import pandas as pd
import pyodbc


In [2]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

**Database connection**

In [3]:

Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



In [4]:
AdventureWorks_queries = {
    'SalesOrderHeader' : 'SELECT * FROM Sales.SalesOrderHeader',
    'SalesOrderDetail' : 'SELECT * FROM Sales.SalesOrderDetail'
}

Northwind_queries = {
    'Orders' : 'SELECT * FROM Orders',
    'OrderDetails' : 'SELECT * FROM [Order Details]'
}

AenC_queries = {
    'sales_order' : 'SELECT * FROM sales_order',
    'sales_order_item' : 'SELECT * FROM sales_order_item',
    'Product' : 'SELECT * FROM Product' 
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_57712\1319449748.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_57712\1319449748.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_57712\1319449748.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table

### Order

**Extract**

In [5]:
#eerst horizontaal samenvoegen per database:
AdventureWorks_Order = pd.merge(SalesOrderHeader, SalesOrderDetail, on = 'SalesOrderID')
Northwind_Order = pd.merge(Orders, OrderDetails, on = 'OrderID')
merge1 = pd.merge(Product, sales_order_item, left_on= 'id', right_on= 'prod_id')
AenC_Order = pd.merge(merge1, sales_order, left_on = 'id_y', right_on = 'id')

print(AdventureWorks_Order.dtypes)

SalesOrderID                       int64
RevisionNumber                     int64
OrderDate                 datetime64[ns]
DueDate                   datetime64[ns]
ShipDate                  datetime64[ns]
Status                             int64
OnlineOrderFlag                     bool
SalesOrderNumber                  object
PurchaseOrderNumber               object
AccountNumber                     object
CustomerID                         int64
SalesPersonID                    float64
TerritoryID                        int64
BillToAddressID                    int64
ShipToAddressID                    int64
ShipMethodID                       int64
CreditCardID                     float64
CreditCardApprovalCode            object
CurrencyRateID                   float64
SubTotal                         float64
TaxAmt                           float64
Freight                          float64
TotalDue                         float64
Comment                           object
rowguid_x       

**Transform**

In [6]:
AdventureWorks_Order.rename(columns={
    'SalesOrderID': 'OrderID',
    'SalesOrderDetailID': 'OrderDetailID',
    'OrderQty': 'Quantity',
    'UnitPriceDiscount': 'Discount',
    'Order_Date': 'OrderDate',
    'SalesPersonID' : 'EmployeeID'
}, inplace=True)

# Vul NaN waarden in EmployeeID met een standaardwaarde (bijv. 0)
AdventureWorks_Order['EmployeeID'].fillna(0, inplace=True)

# change employe id type to int

AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].astype(int)

# change shipvia to string 
#Northwind_Order['ShipVia'] = Northwind_Order['ShipVia'].astype(str)
Northwind_Order.rename(columns={
    'ShipVia': 'ShipMethodID'
}, inplace=True)

AenC_Order.rename(columns={
    'id': 'OrderID',
    'item_id': 'OrderDetailID',
    'quantity_y': 'Quantity',
    'order_date': 'OrderDate',
    'unit_price': 'UnitPrice',
    'cust_id': 'CustomerID',
    'sales_rep': 'EmployeeID',
    'prod_id': 'ProductID'
}, inplace=True)

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_57712\2543201151.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AdventureWorks_Order['EmployeeID'].fillna(0, inplace=True)


**Check of orderID's conflicten**

In [7]:
order_order_ids = set(AdventureWorks_Order['OrderID'])
order_details_order_ids = set(Northwind_Order['OrderID'])
Order_AenC_ids = set(AenC_Order['OrderID'])

# Zoek conflicten tussen AdventureWorks en Northwind
conflicting_order_ids_1 = order_order_ids.intersection(order_details_order_ids)

# Zoek conflicten tussen AdventureWorks en AenC
conflicting_order_ids_2 = order_order_ids.intersection(Order_AenC_ids)

# Zoek conflicten tussen Northwind en AenC
conflicting_order_ids_3 = order_details_order_ids.intersection(Order_AenC_ids)

# Combineer alle conflicterende ID's
all_conflicting_order_ids = conflicting_order_ids_1.union(conflicting_order_ids_2).union(conflicting_order_ids_3)

if all_conflicting_order_ids:
    print("Conflicting Order IDs found:", all_conflicting_order_ids)
else:
    print("No conflicts found in Order IDs.")

# Display the conflicting Order IDs (if any)
conflicts_df = pd.DataFrame(list(all_conflicting_order_ids), columns=['Conflicting Order IDs'])
print(conflicts_df)

No conflicts found in Order IDs.
Empty DataFrame
Columns: [Conflicting Order IDs]
Index: []


**Alle dubbele ID's met een suffix zetten per database**

In [8]:


# Prefixes toevoegen
AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].apply(lambda x: f'AW_{x}')
Northwind_Order['EmployeeID'] = Northwind_Order['EmployeeID'].apply(lambda x: f'NW_{x}')
AenC_Order['EmployeeID'] = AenC_Order['EmployeeID'].apply(lambda x: f'AC_{x}')

# Vervang 'AW_0' met 'NULL' in de kolom EmployeeID
AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].replace('AW_0', None)

AdventureWorks_Order['ShipMethodID'] = AdventureWorks_Order['ShipMethodID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ShipMethodID'] = Northwind_Order['ShipMethodID'].apply(lambda x: f'NW_{x}')
# AenC_Order heeft geen ShipMethodID

AdventureWorks_Order['ProductID'] = AdventureWorks_Order['ProductID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ProductID'] = Northwind_Order['ProductID'].apply(lambda x: f'NW_{x}')
AenC_Order['ProductID'] = AenC_Order['ProductID'].apply(lambda x: f'AC_{x}')

AdventureWorks_Order['CustomerID'] = AdventureWorks_Order['CustomerID'].apply(lambda x: f'AW_{x}')
Northwind_Order['CustomerID'] = Northwind_Order['CustomerID'].apply(lambda x: f'NW_{x}')
AenC_Order['CustomerID'] = AenC_Order['CustomerID'].apply(lambda x: f'AC_{x}')

In [9]:
# Zorg ervoor dat alle vereiste kolommen bestaan en selecteer ze
required_columns = ['OrderID', 'OrderDetailID', 'Quantity', 'UnitPrice', 'Discount', 'OrderDate', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ProductID']

for col in required_columns:
    if col not in AdventureWorks_Order.columns:
        AdventureWorks_Order[col] = None
    if col not in Northwind_Order.columns:
        Northwind_Order[col] = None
    if col not in AenC_Order.columns:
        AenC_Order[col] = None

# Selecteer de kolommen in de juiste volgorde
AdventureWorks_Order = AdventureWorks_Order[required_columns]
Northwind_Order = Northwind_Order[required_columns]
AenC_Order = AenC_Order[required_columns]

# Combineer de dataframes
Order = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)

# Converteer 'Quantity' en 'UnitPrice' naar numerieke typen
Order['Quantity'] = pd.to_numeric(Order['Quantity'], errors='coerce')
Order['UnitPrice'] = pd.to_numeric(Order['UnitPrice'], errors='coerce')

Order['Omzet'] = Order['UnitPrice'] * Order['Quantity']

# Toon het resultaat
#print(combined_df)
print(Order.dtypes)

OrderID           object
OrderDetailID     object
Quantity           int64
UnitPrice        float64
Discount         float64
OrderDate         object
CustomerID        object
EmployeeID        object
ShipMethodID      object
ProductID         object
Omzet            float64
dtype: object


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_57712\1983841409.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Order = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)


In [10]:
# Ensure correct data types and formats
Order['OrderDate'] = pd.to_datetime(Order['OrderDate'], errors='coerce')

# Insert data into the table
for index, row in Order.iterrows():
    try:
        query = """
        INSERT INTO [Order] (OrderID, OrderDetailID, Quantity, UnitPrice, Discount, Omzet, OrderDate, CustomerID, EmployeeID, ShipMethodID, ProductID)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Ensure values are in the correct format
        params = (
            row['OrderID'],
            row['OrderDetailID'] if pd.notnull(row['OrderDetailID']) else None,
            row['Quantity'] if pd.notnull(row['Quantity']) else None,
            float(row['UnitPrice']) if pd.notnull(row['UnitPrice']) else None,
            float(row['Discount']) if pd.notnull(row['Discount']) else None,
            float(row['Omzet']) if pd.notnull(row['Omzet']) else None,
            row['OrderDate'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(row['OrderDate']) else None,
            row['CustomerID'],
            row['EmployeeID'] if pd.notnull(row['EmployeeID']) else None,
            row['ShipMethodID'] if pd.notnull(row['ShipMethodID']) else None,
            row['ProductID']
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)
        print("Parameters:", params)


export_conn.commit()

### **Data_mutatie simuleren**

In [ ]:
# Function to merge data into Order table
def merge_order(order_id, order_detail_id, quantity, unit_price, discount, omzet, order_date, customer_id, employee_id, ship_method_id, product_id):
    merge_query = "{CALL MergeOrder (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)}"
    export_cursor.execute(merge_query, (order_id, order_detail_id, quantity, unit_price, discount, omzet, order_date, customer_id, employee_id, ship_method_id, product_id))
    export_conn.commit()
    print("Merged into Order table")

In [ ]:
# Example usage
try:

    # Merge into Order table
    merge_order('O123', 'OD456', 10, 100.0, 5.0, '1000', '2023-06-07', 'C123', 'E123', 'AW_5', 'P123')
    merge_order('O123', 'OD456', 12, 120.0, 4.0, '1200', '2023-06-07', 'C123', 'E123', 'AW_5', 'P123')

finally:
    # Close the connection
    print('connection closed')